In [0]:
df = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load("/Volumes/pyspark_dbt_project/source/source_data/customers/")

In [0]:
schema_customers = df.schema
schema_customers

In [0]:
display(df.limit(3))

### Spark Streaming ###

In [0]:
entities = ['customers', 'trips', 'locations', 'payments', 'vehicles', 'drivers']

In [0]:
for entity in entities:

        df_batch = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"/Volumes/pyspark_dbt_project/source/source_data/{entity}/")

        schema_entity = df_batch.schema
                
        df = spark.readStream.format("csv")\
                .option("header", True)\
                .schema(schema_entity)\
                .load(f"/Volumes/pyspark_dbt_project/source/source_data/{entity}/")

        # checkpoints helps spark know which files have been processed
        df.writeStream.format("delta")\
        .outputMode("append")\
        .option("checkpointLocation", f"/Volumes/pyspark_dbt_project/bronze/checkpoint/{entity}")\
        .trigger(once=True)\
        .toTable(f"pyspark_dbt_project.bronze.{entity}")